In [1]:
!pip install pokerlib==2.2.6
#https://pypi.org/project/pokerlib/


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: C:\Users\samue\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
from pokerlib import HandParser
from pokerlib.enums import Rank, Suit
import sys
sys.path.append('../pokerlib')

from random import sample
from itertools import product, combinations
from pokerlib import HandParser
from pokerlib.enums import Rank, Suit
import copy

hand1 = HandParser([
    (Rank.KING, Suit.SPADE),
    (Rank.ACE, Suit.SPADE)
])

hand2 = HandParser([
    (Rank.FIVE, Suit.CLUB),
    (Rank.ACE, Suit.CLUB)
])

board = [
    (Rank.EIGHT, Suit.SPADE),
    (Rank.TEN, Suit.SPADE),
    (Rank.JACK, Suit.SPADE),
    (Rank.QUEEN, Suit.SPADE),
    (Rank.TWO, Suit.HEART)
]

# add new cards to each hand
hand1 += board # add the board to hand1
hand2 += board # add the board to hand2

print(hand1.handenum) # Hand.STRAIGHTFLUSH
print(hand2.handenum) # Hand.STRAIGHTFLUSH
print(hand1 > hand2) # True
print(max((Rank.KING, Suit.SPADE), (Rank.ACE, Suit.SPADE))[0])


8
0
True
12


In [ ]:

hands = []
c = list(product(Rank, Suit))
hands = list(combinations(c, 2))
hands = [HandParser(h) for h in hands]
order = sorted(hands)
          

In [2]:
def ranker(cards, board):
    beaten = 0
    c = list(product(Rank, Suit))
    
    for impossible in board + cards:
        c.remove(impossible)
    
    hands = list(combinations(c, 2))
    hand = HandParser(cards)
    hand += board
    for h in hands:
        h = HandParser(list(h))
        h += board
        if h > hand:
            beaten += 1
        
    return beaten, len(hands)
        
    

In [65]:
cards = [
    (Rank.KING, Suit.SPADE),
    (Rank.ACE, Suit.SPADE)
]


# board = [
#     (Rank.EIGHT, Suit.SPADE),
#     (Rank.TEN, Suit.SPADE),
#     (Rank.JACK, Suit.SPADE)
# ]

board= []

print(ranker(cards, board))

(72, 1225)


In [53]:
def move(cards, board, stack, pot, aggressivity_index):
    rank, hands = ranker(cards, board)
    odds = 1 - rank/hands
    print(odds)
    tl = -pot + 2*stack*odds - stack
    tr = (pot**2 + 2*pot*stack + stack**2*(1-2*odds)**2)**0.5
    kelly_solutions = (tl +tr)/2, (tl-tr)/2
    if kelly_solutions[0] + kelly_solutions[1] > 0:
        print(kelly_solutions)
        return kelly_solutions[0]
        
    
move([(Rank.SEVEN, Suit.CLUB),(Rank.TWO, Suit.SPADE)], [], 500, 10, 5)



0.12489795918367352


In [ ]:
def move(cards, board, stack, pot, aggressivity_index):
    rank, hands = ranker(cards, board)
    odds = 1 - rank/hands
    print(odds)
    odds = 0.6
    tl = -pot + 2*stack*odds - stack
    tr = (pot**2 + 2*pot*stack + stack**2*(1-2*odds)**2)**0.5
    kelly_solutions = (tl +tr)/2, (tl-tr)/2
    if kelly_solutions[0] + kelly_solutions[1] > 0:
        print(kelly_solutions)
        return kelly_solutions[0]
        
    
move([(Rank.SEVEN, Suit.CLUB),(Rank.TWO, Suit.SPADE)], [], 100, 0, 5)


0.12489795918367352
(19.999999999999996, 3.552713678800501e-15)


19.999999999999996

: 

120


In [ ]:


def winningProbability(players_cards, board=[], n=1000):
    cards = list(product(Rank, Suit))
    for card in board: cards.remove(card)
    for player_cards in players_cards:
        for card in player_cards:
            cards.remove(card)

    wins = [0] * len(players_cards)
    for i in range(n):
        board_ = sample(cards, 5-len(board))
        hands = [
            HandParser(player_cards + board + board_)
            for player_cards in players_cards
        ]
        winner = max(hands)
        for i, hand in enumerate(hands):
            if hand == winner: wins[i] += 1

    return [win / n for win in wins]


rankings = {}


# for hand in hands:
#     winning = 0
#     nonIncludedCards = copy.copy(cards)
#     print(len(nonIncludedCards))
#     nonIncludedCards.remove(hand[0])
#     nonIncludedCards.remove(hand[1])
#     nonIncludehands = list(combinations(nonIncludedCards, 2))
#     for h in nonIncludehands:
#         winning += winningProbability([list(hand), list(h)])[0]
#     rankings[hand] = winning/1225
    
    
# rankings
    
            



In [6]:
w1, w2 = winningProbability([
    [(Rank.ACE, Suit.HEART), (Rank.KING, Suit.HEART)],
    [(Rank.KING, Suit.SPADE), (Rank.KING, Suit.DIAMOND)]
])
print(w1,w2)
w1, w2 = winningProbability([
    [(Rank.ACE, Suit.HEART), (Rank.KING, Suit.HEART)],
    [(Rank.KING, Suit.SPADE), (Rank.KING, Suit.DIAMOND)]
], [],300)

print(w1, w2)

0.336 0.672
0.3466666666666667 0.6533333333333333


In [24]:
from pokerlib import Table, Player, PlayerSeats
from pokerlib.enums import RoundPublicInId, TablePublicInId


# table that prints outputs
class MyTable(Table):
    def publicOut(self, out_id, **kwargs):
        print(out_id, kwargs)
    def privateOut(self, player_id, out_id, **kwargs):
        print(out_id, kwargs)

# define a new table
table = MyTable(
    _id = 0,
    seats = PlayerSeats([None] * 9),
    buyin = 100,
    small_blind = 5,
    big_blind = 10
)

player1 = Player(
    table_id = table.id,
    _id = 1,
    name = 'alice',
    money = table.buyin
)
player2 = Player(
    table_id = table.id,
    _id = 2,
    name = 'bob',
    money = table.buyin
)
# seat player1 at the first seat
table += player1, 0
# seat player2 at the first free seat
table += player2



table.publicIn(player1.id, TablePublicInId.STARTROUND)
table.publicIn(player1.id, RoundPublicInId.CALL)
print(table.round.board)
table.round.pot_size[0]


0 {'player_id': 1, 'player_seat': 0}
0 {'player_id': 2, 'player_seat': 1}
2 {'round_id': None}
0 {'cards': ([<Rank.ACE: 12>, <Suit.HEART: 3>], [<Rank.ACE: 12>, <Suit.CLUB: 1>])}
0 {'cards': ([<Rank.TEN: 8>, <Suit.CLUB: 1>], [<Rank.SIX: 4>, <Suit.CLUB: 1>])}
0 {}
1 {'turn': <Turn.PREFLOP: 0>, 'board': []}
2 {'player_id': 1, 'paid_amount': 5, 'small_blind': 5}
3 {'player_id': 2, 'paid_amount': 10, 'big_blind': 10}
12 {'player_id': 1, 'to_call': 5}
5 {'player_id': 1, 'paid_amount': 5}
12 {'player_id': 2, 'to_call': 0}
[]


20

In [56]:

[tuple(card) for card in player2.cards]
table.round.board

[[<Rank.FIVE: 3>, <Suit.CLUB: 1>],
 [<Rank.NINE: 7>, <Suit.HEART: 3>],
 [<Rank.SEVEN: 5>, <Suit.SPADE: 0>],
 [<Rank.THREE: 1>, <Suit.HEART: 3>],
 [<Rank.EIGHT: 6>, <Suit.HEART: 3>]]

In [38]:
move([tuple(card) for card in player2.cards], table.round.board, player1.money, table.round.pot_size[0], 5)

In [47]:
table.publicIn(player2.id, RoundPublicInId.CHECK)

4 {'player_id': 2}
1 {'turn': <Turn.RIVER: 3>, 'board': [[<Rank.FIVE: 3>, <Suit.CLUB: 1>], [<Rank.NINE: 7>, <Suit.HEART: 3>], [<Rank.SEVEN: 5>, <Suit.SPADE: 0>], [<Rank.THREE: 1>, <Suit.HEART: 3>], [<Rank.EIGHT: 6>, <Suit.HEART: 3>]]}
12 {'player_id': 1, 'to_call': 0}


In [48]:
table.publicIn(player1.id, RoundPublicInId.CHECK)

4 {'player_id': 1}
12 {'player_id': 2, 'to_call': 0}


In [54]:
move([tuple(card) for card in player2.cards], [tuple(card) for card in table.round.board], player1.money, table.round.pot_size[0], 5)

0.9434343434343434
(36.27882477090172, -20.804077296154244)


36.27882477090172

In [44]:
table.publicIn(player2.id, RoundPublicInId.CHECK)

4 {'player_id': 2}
1 {'turn': <Turn.TURN: 2>, 'board': [[<Rank.FIVE: 3>, <Suit.CLUB: 1>], [<Rank.NINE: 7>, <Suit.HEART: 3>], [<Rank.SEVEN: 5>, <Suit.SPADE: 0>], [<Rank.THREE: 1>, <Suit.HEART: 3>]]}
12 {'player_id': 1, 'to_call': 0}


In [50]:
table.publicIn(player2.id, RoundPublicInId.RAISE, raise_by=50)

7 {'player_id': 2, 'raised_by': 50, 'paid_amount': 50}
12 {'player_id': 1, 'to_call': 50}


In [51]:
table.publicIn(player1.id, RoundPublicInId.CALL)

5 {'player_id': 1, 'paid_amount': 50}
16 {'player_id': 2, 'money_won': 120, 'cards': ([<Rank.TEN: 8>, <Suit.CLUB: 1>], [<Rank.SIX: 4>, <Suit.CLUB: 1>]), 'handname': <Hand.STRAIGHT: 4>, 'hand': [[<Rank.TEN: 8>, <Suit.CLUB: 1>], [<Rank.NINE: 7>, <Suit.HEART: 3>], [<Rank.EIGHT: 6>, <Suit.HEART: 3>], [<Rank.SEVEN: 5>, <Suit.SPADE: 0>], [<Rank.SIX: 4>, <Suit.CLUB: 1>]]}
14 {'player_id': 2, 'cards': ([<Rank.TEN: 8>, <Suit.CLUB: 1>], [<Rank.SIX: 4>, <Suit.CLUB: 1>]), 'kickers': None}
13 {'player_id': 1}
17 {}
